In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.tree import DecisionTreeRegressor

In [2]:
df = pd.read_csv("card_cust.csv")
df.head(2)

CUST_ID      BALANCE  BALANCE_FREQUENCY  PURCHASES  ONEOFF_PURCHASES   
0    10001    40.900749           0.818182       95.4               0.0  \
1    10002  3202.467416           0.909091        0.0               0.0   

   INSTALLMENTS_PURCHASES  CASH_ADVANCE  PURCHASES_FREQUENCY   
0                    95.4      0.000000             0.166667  \
1                     0.0   6442.945483             0.000000   

   ONEOFF_PURCHASES_FREQUENCY  PURCHASES_INSTALLMENTS_FREQUENCY   
0                         0.0                          0.083333  \
1                         0.0                          0.000000   

   CASH_ADVANCE_FREQUENCY  CASH_ADVANCE_TRX  PURCHASES_TRX  CREDIT_LIMIT   
0                    0.00               0.0            2.0        1000.0  \
1                    0.25               4.0            0.0        7000.0   

      PAYMENTS  MINIMUM_PAYMENTS  PRC_FULL_PAYMENT  TENURE  
0   201.802084        139.509787          0.000000    12.0  
1  4103.032597       1072.340217          0.222222    12.0

### 전처리

In [3]:
df.isna().sum()

CUST_ID                              0
BALANCE                              0
BALANCE_FREQUENCY                    0
PURCHASES                            0
ONEOFF_PURCHASES                     0
INSTALLMENTS_PURCHASES               0
CASH_ADVANCE                         0
PURCHASES_FREQUENCY                  0
ONEOFF_PURCHASES_FREQUENCY           0
PURCHASES_INSTALLMENTS_FREQUENCY     0
CASH_ADVANCE_FREQUENCY               0
CASH_ADVANCE_TRX                     0
PURCHASES_TRX                        0
CREDIT_LIMIT                         0
PAYMENTS                             0
MINIMUM_PAYMENTS                    74
PRC_FULL_PAYMENT                     0
TENURE                               0
dtype: int64

In [4]:
df["MINIMUM_PAYMENTS"] = df["MINIMUM_PAYMENTS"].fillna(df["MINIMUM_PAYMENTS"].mean())

In [5]:
df_base = df.copy()

### Q1.

In [6]:
df_q1 = df_base[["TENURE", "BALANCE", "CREDIT_LIMIT"]].copy()
df_q1.head(2)

TENURE      BALANCE  CREDIT_LIMIT
0    12.0    40.900749        1000.0
1    12.0  3202.467416        7000.0

In [7]:
df_q1["TENURE"].unique()

array([12.,  8., 11.,  9., 10.,  7.,  6.])

In [8]:
df_q1.loc[df_q1["TENURE"] == 12, ["BALANCE", "CREDIT_LIMIT"]].corr()

BALANCE  CREDIT_LIMIT
BALANCE       1.000000      0.460833
CREDIT_LIMIT  0.460833      1.000000

In [9]:
df_q1.groupby("TENURE")[["BALANCE", "CREDIT_LIMIT"]].corr()

BALANCE  CREDIT_LIMIT
TENURE                                     
6.0    BALANCE       1.000000      0.868056
       CREDIT_LIMIT  0.868056      1.000000
7.0    BALANCE       1.000000      0.948405
       CREDIT_LIMIT  0.948405      1.000000
8.0    BALANCE       1.000000      0.820696
       CREDIT_LIMIT  0.820696      1.000000
9.0    BALANCE       1.000000      0.085474
       CREDIT_LIMIT  0.085474      1.000000
10.0   BALANCE       1.000000      0.291482
       CREDIT_LIMIT  0.291482      1.000000
11.0   BALANCE       1.000000      0.380360
       CREDIT_LIMIT  0.380360      1.000000
12.0   BALANCE       1.000000      0.460833
       CREDIT_LIMIT  0.460833      1.000000

In [10]:
df_q1.groupby("TENURE")[["BALANCE", "CREDIT_LIMIT"]].corr()["CREDIT_LIMIT"].round(2)

TENURE              
6.0     BALANCE         0.87
        CREDIT_LIMIT    1.00
7.0     BALANCE         0.95
        CREDIT_LIMIT    1.00
8.0     BALANCE         0.82
        CREDIT_LIMIT    1.00
9.0     BALANCE         0.09
        CREDIT_LIMIT    1.00
10.0    BALANCE         0.29
        CREDIT_LIMIT    1.00
11.0    BALANCE         0.38
        CREDIT_LIMIT    1.00
12.0    BALANCE         0.46
        CREDIT_LIMIT    1.00
Name: CREDIT_LIMIT, dtype: float64

In [11]:
df_q1.groupby("TENURE")[["BALANCE", "CREDIT_LIMIT"]].corr().xs("BALANCE", level = 1)

BALANCE  CREDIT_LIMIT
TENURE                       
6.0         1.0      0.868056
7.0         1.0      0.948405
8.0         1.0      0.820696
9.0         1.0      0.085474
10.0        1.0      0.291482
11.0        1.0      0.380360
12.0        1.0      0.460833

In [12]:
df_q1.groupby("TENURE").corr().xs("BALANCE",level = 1)["CREDIT_LIMIT"].round(2).max()

0.95

### Q2.

In [15]:
# df_base.columns = df_base.columns.str.lower()
df_q2 = df_base.drop(columns = "CUST_ID")

In [16]:
df_q2.head(2)

BALANCE  BALANCE_FREQUENCY  PURCHASES  ONEOFF_PURCHASES   
0    40.900749           0.818182       95.4               0.0  \
1  3202.467416           0.909091        0.0               0.0   

   INSTALLMENTS_PURCHASES  CASH_ADVANCE  PURCHASES_FREQUENCY   
0                    95.4      0.000000             0.166667  \
1                     0.0   6442.945483             0.000000   

   ONEOFF_PURCHASES_FREQUENCY  PURCHASES_INSTALLMENTS_FREQUENCY   
0                         0.0                          0.083333  \
1                         0.0                          0.000000   

   CASH_ADVANCE_FREQUENCY  CASH_ADVANCE_TRX  PURCHASES_TRX  CREDIT_LIMIT   
0                    0.00               0.0            2.0        1000.0  \
1                    0.25               4.0            0.0        7000.0   

      PAYMENTS  MINIMUM_PAYMENTS  PRC_FULL_PAYMENT  TENURE  
0   201.802084        139.509787          0.000000    12.0  
1  4103.032597       1072.340217          0.222222    12.0

In [18]:
arr_q2_nor = StandardScaler().fit_transform(df_q2)
arr_q2_nor[:2, ]

array([[-0.84876759, -0.41987944, -0.4419358 , -0.3740477 , -0.39530091,
        -0.48235389, -0.87270064, -0.80432139, -0.71961989, -0.68470093,
        -0.45791753, -0.56411617, -1.16166861, -0.55739615, -0.44372465,
        -0.46554357,  0.28242902],
       [ 0.28279099,  0.01213096, -0.4690169 , -0.3740477 , -0.47030353,
         1.87846763, -1.28255811, -0.80432139, -0.92440303,  0.51349318,
         0.06541679, -0.6280568 ,  0.15002495,  0.36057402, -0.08615941,
         0.33159169,  0.28242902]])

In [ ]:
ls_k = [2, 3, 4, 5]
ls_k = range(2, 6)

ls_sil = []
for k in ls_k:
    model_km = KMeans(n_clusters = k, random_state = 1234)
    model_km.fit(arr_q2_nor)
    val_sil = silhouette_score(arr_q2_nor, labels = model_km.labels_)
    ls_sil = ls_sil + [val_sil]

In [20]:
ls_sil

[0.3075281530456079,
 0.19636128772937608,
 0.207150984946399,
 0.19274056144483248]

In [22]:
best_k = pd.Series(ls_sil, index = ls_k).idxmax()
best_k

2

In [ ]:
model_km_best = KMeans(n_clusters = best_k, random_state = 1234)
model_km_best.fit(arr_q2_nor)

In [30]:
df_q2["cluster"] = model_km_best.labels_

In [31]:
df_q2.groupby("cluster")["ONEOFF_PURCHASES"].mean()

cluster
0     340.230998
1    3946.187525
Name: ONEOFF_PURCHASES, dtype: float64

In [32]:
df_q2.groupby("cluster")["ONEOFF_PURCHASES"].mean().round(2).max()

3946.19

### Q3.

In [33]:
12 % 4, 11 % 4

(0, 3)

In [34]:
df_train = df_base.loc[(df_base["CUST_ID"] % 4) != 0, ]
df_test  = df_base.loc[(df_base["CUST_ID"] % 4) == 0, ]

In [37]:
col_rm = ["CUST_ID", "ONEOFF_PURCHASES"]

model_dt = DecisionTreeRegressor(random_state = 1234)
model_dt.fit(X = df_train.drop(columns = col_rm),
             y = df_train["ONEOFF_PURCHASES"])
pred = model_dt.predict(df_test.drop(columns = col_rm))
pred[:4]

array([1500.,    0., 1490.,    0.])

In [38]:
from sklearn.metrics import mean_squared_error

In [39]:
round(mean_squared_error(y_true = df_test["ONEOFF_PURCHASES"],
                         y_pred = pred) ** 0.5, 1)

1039.2

In [40]:
y_t = df_test["ONEOFF_PURCHASES"],
y_p = pred

In [45]:
round(((y_t - y_p) ** 2).mean() ** 0.5, 1)

1039.2